## Group 15 TM names : 
- Ahmed   : 23PGAI0120
- Akash Deshwani: 23PGAI0035
- Harshada Suresh Jadhav: 23PGAI0101
- Rohan Mehta: 23PGAI0001

## Installing the required packages

In [1]:
!pip install bs4
!pip install numpy 
!pip install pandas
!pip install requests

  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached beautifulsoup4-4.11.2-py3-none-any.whl (129 kB)
  Running setup.py install for bs4: started
  Running setup.py install for bs4: finished with status 'done'


  DEPRECATION: bs4 is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559


  Using cached pandas-1.5.3-cp310-cp310-win_amd64.whl (10.4 MB)
  Using cached pytz-2022.7.1-py2.py3-none-any.whl (499 kB)
  Using cached requests-2.28.2-py3-none-any.whl (62 kB)
  Using cached urllib3-1.26.14-py2.py3-none-any.whl (140 kB)
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
     ---------------------------------------- 0.0/96.5 kB ? eta -:--:--
     ---------------------------------------- 96.5/96.5 kB 5.4 MB/s eta 0:00:00


## Importing the required packages

In [2]:
import json
import numpy as np
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import csv

#### Importing the dataset from Json file

In [3]:
def read_file():
    a = "gg2015.json"
    df_original = pd.read_json(a)
    print(len(df_original))
    
    # Creating a copy of the dataframe for manipulating
    df = df_original.copy()
    
    # Creating a list of text for finding hosts
    text_host = df['text'].tolist()
    return(text_host)

In [13]:
# df_test = read_file()
# print(type(df_test))
# print(df_test[:5])

#### Preprocessing the data and converting it into a required list 

In [6]:
# Preprocessing data
def preprocessing(data):
    processed_data = []
    remove_list = ['think', 'thinking', 'should', 'would', 'maybe', 'could']
    for i in data:
        for j in remove_list:
            if j not in i:
                processed_data.append(i)
    remove_words_list = ['golden', 'globe', 'globes']
    for i in range(len(processed_data)):
        for j in remove_words_list:
            processed_data[i] = re.sub(j, '', processed_data[i], flags = re.IGNORECASE)
    return processed_data

In [7]:
# df_1 = preprocessing(df_test)
# print(df_1[:5])
# print(len(df_1))

#### Scraping the Top 1000 Actors and Actresses from the IMDB website

In [8]:
def scrape_actors():
    actorNames = []
    for single_page in range(1, 11):
        URL = f"https://www.imdb.com/list/ls058011111/?sort=list_order,asc&mode=detail&page={single_page}"
        r = requests.get(URL)
        soup = BeautifulSoup(r.text, 'html.parser')
        actorName = soup.find_all('h3',{'class':'lister-item-header'})
        actorName = [i.find('a').text.strip() for i in actorName]
        actorNames.extend(actorName)
    return actorNames

In [9]:
# scrape_list = scrape_actors()
# print(scrape_list[:5])
# print(len(scrape_list))

### Extracting Hosts name from the Tweet text

In [10]:
# Finding Hosts
def get_hosts(data, actorNames):
    match = ''
    result={}
    for i in range(len(data)):
        found = re.search('(H|h)ost', data[i])
        if found:
            patt = re.compile(r'[A-Z][a-z]+ [A-Z][a-z]+')
            matches = patt.findall(data[i])
            for match in matches:
                if match in actorNames:
                    if match in result:
                        result[match] += 1
                    else:
                        result[match] = 1
    sorted_hosts = sorted(result.items(), key=lambda x:x[1], reverse=True)
    return(sorted_hosts[0:2])

In [11]:
# host = get_hosts(df_1, scrape_list)
# print(host)

### Hurrah! We have successfully extracted the hosts name from the tweet text

##### Extracting the awards from the tweet text and storing it in a list

In [12]:
def get_awards(data):
    match = ''
    result = {}
    for i in range(len(data)):
        found = re.search('(B|b)est', data[i])
        if found:
            found1 = re.search(r'\bWin\b | \bWins\b | \bwin\b | \bwins\b | \bgoes to\b|', data[i])
            if found1:
                patt = re.compile(r'Best [A-Z][a-z]+ [A-Z*a-z ]+ - [A-Z*a-z ]+')
                matches = patt.findall(data[i])
                for match in matches:
                    if match.title() in result:
                        result[match.title()] += 1
                    else:
                        result[match.title()] = 1
    sorted_awards = sorted(result.items(), key=lambda x:x[1], reverse=True)
    return(sorted_awards[0:50])

In [13]:
# awards = get_awards(df_1)
# print(awards[:5])
# print(len(awards))

##### we have found the awards but it is not in the required format and it has some extra characters (noise) in it

### Searching for the Presenters name in the tweet text

In [14]:
# Finding Presenters
def get_presenters(data, actorNames):
    match = ''
    result={}
    for i in range(len(data)):
        found = re.search('(P|p)resent(ing|ed|er|ers)', data[i])
        if found:
            patt = re.compile(r'[A-Z][a-z]+ [A-Z][a-z]+')
            matches = patt.findall(data[i])
            for match in matches:
                if match in actorNames:
                    if match in result:
                        result[match] += 1
                    else:
                        result[match] = 1
    sorted_presenters = sorted(result.items(), key=lambda x:x[1], reverse=True)
    return(sorted_presenters[0:15])

In [15]:
# presenters = get_presenters(df_1, scrape_list)
# print(presenters[:5])
# print(len(presenters))

#### Nominees for all the awards

In [16]:
# Finding Nominees
def get_nominees(data, actorNames):
    match = ''
    result={}
    for i in range(len(data)):
        found = re.search('(N|n)omin(ee|ees|ated)', data[i])
        if found:
            patt = re.compile(r'[A-Z][a-z]+ [A-Z][a-z]+')
            matches = patt.findall(data[i])
            for match in matches:
                if match in actorNames:
                    if match in result:
                        result[match] += 1
                    else:
                        result[match] = 1
    sorted_presenters = sorted(result.items(), key=lambda x:x[1], reverse=True)
    return(sorted_presenters[0:100])

In [17]:
# nom = get_nominees(df_1, scrape_list)
# print(nom[:5])
# print(len(nom))

#### Finally Winners of all the awards

In [18]:
def get_winners(data, awards, actorNames):
    match = ''
    result = {}
    for i in range(len(data)):
        for j in range(len(awards)):
            found = re.search(awards[j][0].lower(), data[i].lower())
            if found:
                found1 = re.search(r'\bWin\b | \bWins\b | \bwin\b | \bwins\b | \bgoes to\b|', data[i])
                if found1:
                    patt = re.compile(r'[A-Z][a-z]+ [A-Z][a-z]+')
                    matches = patt.findall(data[i])
                    for match in matches:
                        if match in actorNames:
                            if match.title() in result:
                                result[match.title()] += 1
                            else:
                                result[match.title()] = 1
    sorted_awards = sorted(result.items(), key=lambda x:x[1], reverse=True)
    return(sorted_awards[0:50])

In [19]:
# winnerslist = get_winners(df_1, awards, scrape_list)
# print(winnerslist[:5])
# print(len(winnerslist))

In [20]:
def report(hosts, awards, presenters, nominees, winners):
    
    print('Hosts: ')
    for i in hosts:
        print(i[0])
    print()
    print('Presenters: ')
    for i in presenters:
        print(i[0])
    print()
    print('Award Names: ')
    for i in awards:
        print(i[0])
    print()
    print('Nominees: ')
    for i in nominees:
        print(i[0])
    print()
    print('Winners: ')
    for i in winners:
        print(i[0])
    print()

In [21]:
text_host = read_file()
text_host1 = preprocessing(text_host)
actorNames = scrape_actors()
hosts = get_hosts(text_host1, actorNames)
awards = get_awards(text_host1)
presenters = get_presenters(text_host1,actorNames)
nominees = get_nominees(text_host1, actorNames)
winners = get_winners(text_host1, awards, actorNames)
report(hosts, awards, presenters, nominees, winners)

1754153
Hosts: 
Amy Poehler
Tina Fey

Presenters: 
Jennifer Aniston
Salma Hayek
Dakota Johnson
Naomi Watts
Benedict Cumberbatch
Meryl Streep
Tina Fey
Katie Holmes
Jennifer Lopez
Kate Hudson
Amy Poehler
Paul Rudd
Bill Hader
Kevin Hart
Jeremy Renner

Award Names: 
Best Actor In A Motion Picture - Drama 
Best Motion Picture - Comedy Or Musical 
Best Supporting Actor In A Television Series - Matt Bomer 
Best Television Series Actor - Drama 
Best Actress In A Motion Picture - Comedy Or Musical 
Best Motion Picture - Drama 
Best Actress In A Motion Picture - Drama 
Best Animated Feature Film - How To Train Your Dragon 
Best Actor In A Motion Picture - Comedy Or Musical 
Best Supporting Actor In A Motion Picture - J
Best Supporting Actress In A Motion Picture - Patricia Arquette  
Best Miniseries Or Motion Picture Made For Television - Fargo 
Best Actor In A Tv Movie Or Miniseries - Billy Bob Thornton  
Best Actress In A Tv Movie Or Miniseries - Maggie Gyllenhaal 
Best Foreign Film - Leviatha

In [25]:
print('Presenters: ')
for i in presenters:
    print(i[0])
print()

Presenters: 
Jennifer Aniston
Salma Hayek
Dakota Johnson
Naomi Watts
Benedict Cumberbatch
Meryl Streep
Tina Fey
Katie Holmes
Jennifer Lopez
Kate Hudson
Amy Poehler
Paul Rudd
Bill Hader
Kevin Hart
Jeremy Renner



In [23]:
print('Presenters: ')
for i in presenters:
    print(i[0])
print()

Presenters: 
Jennifer Aniston
Salma Hayek
Dakota Johnson
Naomi Watts
Benedict Cumberbatch
Meryl Streep
Tina Fey
Katie Holmes
Jennifer Lopez
Kate Hudson
Amy Poehler
Paul Rudd
Bill Hader
Kevin Hart
Jeremy Renner



In [24]:
print('Award Names: ')
for i in awards:
    print(i[0])
print()

Award Names: 
Best Actor In A Motion Picture - Drama 
Best Motion Picture - Comedy Or Musical 
Best Supporting Actor In A Television Series - Matt Bomer 
Best Television Series Actor - Drama 
Best Actress In A Motion Picture - Comedy Or Musical 
Best Motion Picture - Drama 
Best Actress In A Motion Picture - Drama 
Best Animated Feature Film - How To Train Your Dragon 
Best Actor In A Motion Picture - Comedy Or Musical 
Best Supporting Actor In A Motion Picture - J
Best Supporting Actress In A Motion Picture - Patricia Arquette  
Best Miniseries Or Motion Picture Made For Television - Fargo 
Best Actor In A Tv Movie Or Miniseries - Billy Bob Thornton  
Best Actress In A Tv Movie Or Miniseries - Maggie Gyllenhaal 
Best Foreign Film - Leviathan 
Best Television Series - Drama 
Best Actor In A Television Series - Comedy Or Musical 
Best Television Series - Comedy Or Musical 
Best Actress In A Television Series - Drama 
Best Motion Picture - Drama
Best Motion Picture - Musical Or Comedy
Be